In [ ]:
from robotic import ry
import numpy as np
import time

###########################################PART-1 STARTS HERE##########################################

In [ ]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [ ]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])

ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


RRT

In [51]:
#define a path finding problem
rrt = ry.PathFinder()
i = 0
while True:
    i = i + 1
    print(i)
    rrt.setProblem(C, [qHome], [qT])
    ret = rrt.solve()
    path = ret.x
    print("rettttt: ", ret)
    #print("path size:", path)
    # display the path
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        for t in range(0, path.shape[0] - 1):
            C.setJointState(path[t])
            C.view()
            time.sleep(0.002)
        break

1
rettttt:  { time: 0.303077, evals: 4267, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


Trajectory Optimization via KOMO

In [ ]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path for part 1
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)


###########################################PART-1 ENDS HERE##########################################